# Connect to the cohere container database

In [1]:
# Load the .env file
#pip install -U python-dotenv
import os
from dotenv import load_dotenv
load_dotenv(encoding='utf-8')

True

In [3]:
import psycopg2

# Connect to the database
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)

# Create a cursor object
cur = conn.cursor()

# Execute a SQL query
cur.execute("SELECT * FROM citation_documents;")
#cur.execute("SELECT * FROM citation_documents;")

# Fetch the results
rows = cur.fetchall()

# Print the results
for row in rows:
    print(row)

# Close the cursor and connection
cur.close()
conn.close()

In [7]:
import psycopg2

# Connect to the database
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)

# Create a cursor object
cur = conn.cursor()

# Execute a SQL query
cur.execute("SELECT * FROM messages;")

# Fetch the results
rows = cur.fetchall()

# Print the results
for row in rows:
    print(row)

# Close the cursor and connection
cur.close()
conn.close()

('What is the travel reimbursement policy at Tech Innovator Inc?', None, 'USER', '5e5eb1fb-2df0-4b1b-99be-cc9af6768791', '94ee2d56-250a-46cc-b709-2e2c8a6b6da6', 0, True, '6ea2ab3a-4e65-48ed-bc4d-9bf6e9000a0e', datetime.datetime(2024, 8, 23, 7, 32, 30, 82017), datetime.datetime(2024, 8, 23, 7, 32, 30, 82017), None)
('', None, 'CHATBOT', '5e5eb1fb-2df0-4b1b-99be-cc9af6768791', '94ee2d56-250a-46cc-b709-2e2c8a6b6da6', 0, True, 'b8de4b06-1a0e-446b-9c46-8eaeb123c503', datetime.datetime(2024, 8, 23, 7, 33, 1, 577401), datetime.datetime(2024, 8, 23, 7, 33, 1, 577401), None)
('Who wrote the finance documents?', None, 'USER', '5e5eb1fb-2df0-4b1b-99be-cc9af6768791', '0b0881f8-e945-4c5d-9da2-58c0ade75855', 0, True, '60909156-e66a-40dc-ae3a-515faa982a80', datetime.datetime(2024, 8, 23, 15, 34, 59, 257051), datetime.datetime(2024, 8, 23, 15, 34, 59, 257051), None)
("I don't have enough information to answer the question. You haven't provided any context about the finance documents, so I'm unable to 

# Create a test data

In [22]:
# pip install cohere numpy
import numpy as np
import cohere
os.environ["COHERE_API_KEY"] = os.getenv('COHERE_API_KEY')
co = cohere.Client(api_key=os.environ["COHERE_API_KEY"])

d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [23]:
# Define the documents
faqs_long = [
    {"text": "Joining Slack Channels: You will receive an invite via email. Be sure to join relevant channels to stay informed and engaged."},
    {"text": "Finding Coffee Spots: For your caffeine fix, head to the break room's coffee machine or cross the street to the café for artisan coffee."},
    {"text": "Team-Building Activities: We foster team spirit with monthly outings and weekly game nights. Feel free to suggest new activity ideas anytime!"},
    {"text": "Working Hours Flexibility: We prioritize work-life balance. While our core hours are 9 AM to 5 PM, we offer flexibility to adjust as needed."},
    {"text": "Side Projects Policy: We encourage you to pursue your passions. Just be mindful of any potential conflicts of interest with our business."},
    {"text": "Reimbursing Travel Expenses: Easily manage your travel expenses by submitting them through our finance tool. Approvals are prompt and straightforward."},
    {"text": "Working from Abroad: Working remotely from another country is possible. Simply coordinate with your manager and ensure your availability during core hours."},
    {"text": "Health and Wellness Benefits: We care about your well-being and offer gym memberships, on-site yoga classes, and comprehensive health insurance."},
    {"text": "Performance Reviews Frequency: We conduct informal check-ins every quarter and formal performance reviews twice a year."},
    {"text": "Proposing New Ideas: Innovation is welcomed! Share your brilliant ideas at our weekly team meetings or directly with your team lead."},
]

# Embed the documents
doc_emb = co.embed(
            model="embed-english-v3.0",
            input_type="search_document",
            texts=[doc['text'] for doc in faqs_long]).embeddings


In [24]:
# Add the user query
query = "How to get to know my teammates"
# Generate the search query
response = co.chat(message=query,
                  search_queries_only=True)
query_optimized = response.search_queries[0].text
# Embed the search query
query_emb = co.embed(
    model="embed-english-v3.0",
    input_type="search_query",
    texts=[query_optimized]).embeddings

In [25]:
# Compute dot product similarity and display results
n = 5
scores = np.dot(query_emb, np.transpose(doc_emb))[0]
scores_sorted = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)[:n]
retrieved_documents = [faqs_long[item[0]] for item in scores_sorted]
for idx, item in enumerate(scores_sorted):
    print(f"Rank: {idx+1}")
    print(f"Score: {item[1]}")
    print(f"Document: {faqs_long[item[0]]}\n")

Rank: 1
Score: 0.32653470360872655
Document: {'text': 'Team-Building Activities: We foster team spirit with monthly outings and weekly game nights. Feel free to suggest new activity ideas anytime!'}

Rank: 2
Score: 0.2685185535226478
Document: {'text': 'Proposing New Ideas: Innovation is welcomed! Share your brilliant ideas at our weekly team meetings or directly with your team lead.'}

Rank: 3
Score: 0.2581341975304149
Document: {'text': 'Joining Slack Channels: You will receive an invite via email. Be sure to join relevant channels to stay informed and engaged.'}

Rank: 4
Score: 0.18633336738178458
Document: {'text': "Finding Coffee Spots: For your caffeine fix, head to the break room's coffee machine or cross the street to the café for artisan coffee."}

Rank: 5
Score: 0.13022396595682817
Document: {'text': 'Health and Wellness Benefits: We care about your well-being and offer gym memberships, on-site yoga classes, and comprehensive health insurance.'}



In [26]:
# Rerank the documents
results = co.rerank(query=query_optimized,
                    documents=retrieved_documents,
                    top_n=2,
                    model='rerank-english-v3.0')

# Display the reranking results
for idx, result in enumerate(results.results):
    print(f"Rank: {idx+1}") 
    print(f"Score: {result.relevance_score}")
    print(f"Document: {retrieved_documents[result.index]}\n")
    
reranked_documents = [retrieved_documents[result.index] for result in results.results]

Rank: 1
Score: 0.0040072887
Document: {'text': 'Joining Slack Channels: You will receive an invite via email. Be sure to join relevant channels to stay informed and engaged.'}

Rank: 2
Score: 0.0020829707
Document: {'text': 'Team-Building Activities: We foster team spirit with monthly outings and weekly game nights. Feel free to suggest new activity ideas anytime!'}



In [27]:
# Generate the response
response = co.chat(
                message=query_optimized,
                model="command-r-plus",
                documents=reranked_documents)

# Display the response
print(response.text)

# Display the citations and source documents
if response.citations:
  print("\nCITATIONS:")
  for citation in response.citations:
    print(citation)

  print("\nDOCUMENTS:")
  for document in response.documents:
    print(document)

Getting to know your teammates can be done in a variety of ways. You could join your company's Slack channels to stay informed and connected. You could also take part in team-building activities such as outings and game nights.

CITATIONS:
start=75 end=109 text="join your company's Slack channels" document_ids=['doc_0']
start=113 end=141 text='stay informed and connected.' document_ids=['doc_0']
start=170 end=194 text='team-building activities' document_ids=['doc_1']
start=203 end=227 text='outings and game nights.' document_ids=['doc_1']

DOCUMENTS:
{'id': 'doc_0', 'text': 'Joining Slack Channels: You will receive an invite via email. Be sure to join relevant channels to stay informed and engaged.'}
{'id': 'doc_1', 'text': 'Team-Building Activities: We foster team spirit with monthly outings and weekly game nights. Feel free to suggest new activity ideas anytime!'}


# Converting Conversation Data to HuggingFace Dataset object

In [72]:
response_dict = {
    'question':[[query]],
    'answer':[[response.text]],
    'contexts':[["\n".join([document["text"] for document in response.documents])]],
    'ground_truth':[['true']]
}

In [73]:
response_dict

{'question': [['How to get to know my teammates']],
 'answer': [["Getting to know your teammates can be done in a variety of ways. You could join your company's Slack channels to stay informed and connected. You could also take part in team-building activities such as outings and game nights."]],
 'contexts': [['Joining Slack Channels: You will receive an invite via email. Be sure to join relevant channels to stay informed and engaged.\nTeam-Building Activities: We foster team spirit with monthly outings and weekly game nights. Feel free to suggest new activity ideas anytime!']],
 'ground_truth': [['true']]}

In [75]:
from datasets import Dataset
# Define features with variable length for "answer"
#features = {'question': str, 'answer': Dataset.Variable(dtype=str), 'contexts': [Dataset.Feature({'id': str, 'text': str})]}

# Create the dataset with the features
dataset = Dataset.from_dict(response_dict)

# Evaluation with RAGAS

In [21]:
# Define LLM
from langchain_groq import ChatGroq
llm = ChatGroq(
    groq_api_key=os.environ["GROD_CLOUD_API_KEY"],
    model_name='llama-3.1-8b-instant'
    )

#Define Enbeddings
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
fastembed_embeddings = FastEmbedEmbeddings()

d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 5599.87it/s]


In [76]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    context_utilization,
)

from langchain_core.language_models import BaseLanguageModel
from langchain_core.embeddings import Embeddings

result = evaluate(
    dataset = dataset, 
    metrics=[
        #context_precision,
        context_utilization,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
    llm=llm,
    embeddings=fastembed_embeddings, 
    #run_config=RunConfig(max_workers=2),
)

df = result.to_pandas()

ValueError: Dataset feature "question" should be of type string